# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#getting data: the dataset was uploaded to and is accesible through my github repo
#ancla
from azureml.data.dataset_factory import TabularDatasetFactory

rawdata_homic2018 = "https://raw.githubusercontent.com/hftamayo/azuremlproj03/main/cad2018.csv"
dshomic2018 = TabularDatasetFactory.from_delimited_files(path=rawdata_homic2018, separator=',')
hyperpdhomic2018 = dshomic2018.to_pandas_dataframe().dropna()

In [3]:
ws = Workspace.from_config()
expname = 'hyperp3ht'

exphyper=Experiment(ws, expname)

In [4]:
# Compute name should contain only letters, digits, hyphen and should be 2-16 charachters long
ccname = "ccp03hyp"
try:
    ccluster = ComputeTarget(ws, ccname)
    print(f"{ccname} exists already")
except:
    ccconfig = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=5)
    ccluster = ComputeTarget.create(ws, ccname, ccconfig)
ccluster.wait_for_completion(show_output=True)


Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [39]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
#param_sampling = RandomParameterSampling({"--C": choice(1,2,3,4,5), "--max_iter": choice(80,100,120,150,170,200)})
param_sampling = RandomParameterSampling({"--C": choice(1,2,3,4), "--max_iter": choice(40,80,120,130,200)})

if "outputs" not in os.listdir():
    os.mkdir("./outputs")

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='.', compute_target=ccluster, entry_script='train.py')

hyperdrive_run_config = HyperDriveConfig(estimator=estimator, policy=early_termination_policy, primary_metric_name="Accuracy",
                                          hyperparameter_sampling=param_sampling,
                                         max_total_runs=20,
                                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)

In [40]:
#TODO: Submit your experiment
hyperDrive_run = exphyper.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [41]:
RunDetails(hyperDrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [42]:
brmodel = hyperDrive_run.get_best_run_by_primary_metric()
brmetrics = brmodel.get_metrics()

print('Best Run Id: ', brmodel.id)
print('\n Accuracy:', brmetrics['Accuracy'])
print('\n Regularization Strength:',brmetrics['Regularization Strength:'])
print('\n Max Iterations:',brmetrics['Max iterations:'])

Best Run Id:  HD_338bb87e-8b3d-4e92-8511-e8692b49bd83_1

 Accuracy: 0.8914342629482072

 Regularization Strength: 1.0

 Max Iterations: 80


In [62]:
import joblib
#os.makedirs('outputs', exist_ok=True)
joblib.dump(value=brmodel.id, filename='outputs/bhypermodel.joblib')

['outputs/bhypermodel.joblib']

In [63]:
brmodel.upload_file('bhypermodel', 'outputs/bhypermodel.joblib')

In [64]:
#TODO: Save the best model
#ancla
#model = brmodel.register_model(model_name='HyperDrive_HighAccuracy', model_path='bhypermodel.joblib', 
#                                properties={'Accuracy': brmetrics['Accuracy'],
#                                            'Regularization Strength': brmetrics['Regularization Strength:'],
#                                           'Max Iterations': brmetrics['Max iterations:']})
#model = brmodel.register_model(model_name='bhypermodel.joblib', model_path='.')
#model = brmodel.register_model(model_name='bhypermodel', model_path = 'outputs/bhypermodel.joblib')
#model = brmodel.register_model('bhypermodel', model_path = 'outputs/bhypermodel.joblib')
model = brmodel.register_model(model_name = 'bhypermodel', model_path = 'outputs/bhypermodel.joblib') 
print(model)

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path outputs/bhypermodel.joblib in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_7b0b9b80ea9d12a9a728593ce91f165702e3dc963666348969e37d5370962ae5_d.txt', 'azureml-logs/65_job_prep-tvmps_7b0b9b80ea9d12a9a728593ce91f165702e3dc963666348969e37d5370962ae5_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_7b0b9b80ea9d12a9a728593ce91f165702e3dc963666348969e37d5370962ae5_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'bhypermodel', 'logs/azureml/98_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/hyperDrive_1.0_80']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path outputs/bhypermodel.joblib in the set of files uploaded to the r

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [52]:
from azureml.core import Environment
#from azureml.core.model import InferenceConfig
env = Environment.get(workspace=ws, name="AzureML-AutoML")
#OPCIONAL: env = Environment.get(workspace, "AzureML-Minimal").clone(env_name)

In [58]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig, Model

infcfg = InferenceConfig(entry_script='score.py',
                                    environment=env)

# deploying the model via WebService
from azureml.core.webservice import AciWebservice

depcfg = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
webservice = Model.deploy(ws, "ws8hypp03ht01", [model], infcfg, depcfg)
webservice.wait_for_deployment(show_output = True)
print(webservice.state)

print(webservice.scoring_uri)

print(webservice.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..............................................................................
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: a9a27885-6494-41cc-8fca-904b86cb1e4c
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: ws7hypp03ht01. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\nYou can also try to run image viennaglobal.azurecr.io/azureml/azureml_1b9697d50bbbb35eb098c299c7ed3dd0 locally. Please refer to https://aka.ms/debugimage#service-launch-fails for mo

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: a9a27885-6494-41cc-8fca-904b86cb1e4c
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: ws7hypp03ht01. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\nYou can also try to run image viennaglobal.azurecr.io/azureml/azureml_1b9697d50bbbb35eb098c299c7ed3dd0 locally. Please refer to https://aka.ms/debugimage#service-launch-f

In [ ]:
import json

#Importing the data set for testing 
tsthomic2018 = hyperpdhomic2018.sample(10) 
lblhomic2018 = tsthomic2018.pop('sexo')

tstdatahomic2018 = json.dumps({'data': tsthomic2018.to_dict(orient='records')})

print(tstdatahomic2018)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests

headers = {'Content-type': 'application/json'}

response = requests.post(webservice.scoring_uri, tstdatahomic2018, headers=headers)

In [ ]:
# Print results from the inference
print(response.text)

In [ ]:
# Print original labels
print(lblhomic2018)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(webservice.get_logs())

In [ ]:
#Optional: converting the model to ONNX format
brmodel2 = hyperDrive_run.get_output()
from azureml.automl.runtime.onnx_convert import OnnxConverter

brmodel2, onnx_model= hyperDrive_run.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(onnx_model, file_path="./hyperbmodel.onnx")
print(brmodel2)

In [ ]:
#BEST PRACTICES: deleting webservice and CCluster that won't be used anymore
webservice.delete()
ccluster.delete()